In [70]:
import pandas as pd
import numpy as np
import os
import glob

#datadir = "C:/Users/jdhuang/OneDrive - Colorado School of Mines/Research/MIDDMI/TCO/Data"
indir = os.path.join(os.environ['HOME'], 'OneDrive - Colorado School of Mines/Research/MIDDMI/TCO/in')
outdir = os.path.join(os.environ['HOME'], 'OneDrive - Colorado School of Mines/Research/MIDDMI/TCO/out')
logname = 'gen_csv_log.txt'
os.chdir(indir)

#load variable definitions
vardictdf = pd.read_excel('VariableDict.xlsx')

#put standard variables into dict
stdvardf = vardictdf[vardictdf.Keyword.str[0] != '!']
stdvars = list(stdvardf.VarName)
headers = list(stdvardf.Keyword + ': ' + stdvardf.DisplayName)
stdvardict = dict(zip(stdvars,headers))

#composition variables
compvardf = vardictdf[vardictdf.Keyword == '!composition']
compvars = list(compvardf.VarName)

#library prefix to strip out
libprefix = 'PDAC_COM3_'

#log file
log = open(os.path.join(outdir,logname), 'w')

#loop through all directories in indir
for lib in next(os.walk(indir))[1]:
    log.write('Processing library ' + lib + '\n')
    csvname = lib + '_AllVar.csv'
    os.chdir(lib)
    
    all_var = pd.DataFrame([])
    
    #read the Points file to determine file length
    pts = pd.read_csv(lib + '_Points.txt',sep='\t',usecols = [0]) 
    
    #dataframes to store composition data
    chm = pd.DataFrame()#np.zeros((len(pts),1),dtype='str'),columns=['formula'])
    sitesum = pd.DataFrame(np.zeros((len(pts),2)), columns = ['A','B'])
    
    #track found variables
    foundvars = ['Point','Row','Column']
    
    for fname in glob.glob(lib + '*.txt'):
        vname = fname[len(lib)+1:fname.find('.txt')]
        #regular variables
        if vname in stdvars:
            if len(all_var) == 0:
                cols = None
            else:
                cols = [3]
            df = pd.read_csv(fname,sep='\t',usecols = cols)
            all_var = pd.concat([all_var,df], axis=1)
        #composition variables
        elif vname in list(compvar.VarName):
            dfc = pd.read_csv(fname,sep='\t',usecols = [3])
            elmnt = vname[:vname.find('_at')]
            chm[elmnt] = dfc[vname] 
            sitesum.A = sitesum.A + dfc[vname]
        
        foundvars.append(vname)
    
    #identify ignored and missing variables
    ignoredvars = np.setdiff1d(foundvars,stdvars+compvars, assume_unique=True)
    missingstd = np.setdiff1d(stdvars,foundvars, assume_unique=True)
    missingcomp = np.setdiff1d(compvars,foundvars, assume_unique=True)
    log.write('Ignored variables:\n\t' + '\n\t'.join(ignoredvars) + '\n')
    log.write('*Warning: missing standard variables:\n\t' + '\n\t'.join(missingstd) + '\n')
    log.write('*Warning: missing composition variables:\n\t' + '\n\t'.join(missingcomp) + '\n')
        
    #determine composition
    sitesum.B = 1 - sitesum.A
    sitemax = sitesum.max(axis=1)
    sitenorm = sitesum.divide(sitemax,axis=0) #normalize for the higher occupancy site
    chmnorm = chm.divide(sitemax,axis=0)
    chmnorm.insert(0,'Ba',list(sitenorm.B))
    chmnorm = chmnorm.round(5)
    for elmnt in list(chmnorm.columns):
        chmnorm[elmnt] = elmnt + chmnorm[elmnt].map(str) 
    chmnorm['formula'] = chmnorm.apply(lambda x: ''.join(x),axis=1)
    chmnorm['formula'] = chmnorm['formula'] + 'O3'

    sample = lib[len(libprefix):]

    all_var = all_var.rename(index=str, columns = stdvardict) #rename headers to ingestable format using vardict
    all_var.insert(0,'IDENTIFIER: Sample number',sample) #add sample column        
    all_var.insert(0,'FORMULA',list(chmnorm['formula'])) #add formula column
    all_var.to_csv(os.path.join(outdir,csvname),index=False)
    
    log.write('Wrote ' + csvname + '\n')
    log.write('Finished processing library ' + lib + '\n\n')
    os.chdir('..')
    
log.close()

In [25]:
params = pd.read_excel('PLD_params.xlsx',skiprows=1)

In [26]:
params

,Sample #,Date,Targets Loaded,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Target/s used,Laser Energy (mJ),...,# Cycles,Frequency,pO2 (mTorr),Substrate Angle,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Substrate
0,NaN,NaN,Target 1,Target 2,Target 3,Target 4,Target 5,Target 6,NaN,NaN,...,NaN,NaN,NaN,Target 1,Target 2,Target 3,Target 4,Target 5,Target 6,NaN
1,01023,9_Feb_2016,BZC91,-,BZYF631,-,BZF19,-,BZYF631,400.0,...,10.0,10,50.0,-,-,45,-,-,-,Eagle XG
2,01024,10_Feb_2016,BZC91,-,BZYF631,-,BZF19,-,BZYF631,400.0,...,20.0,10,50.0,-,-,90,-,-,-,Eagle XG
3,01025,11_Feb_2016,BZC91,-,BZYF631,-,BZF19,-,BZF19,400.0,...,20.0,10,50.0,-,-,-,-,90,-,Eagle XG
4,01026,12_Feb_2016,BZC91,-,BZYF631,-,BZF19,-,BZC91,400.0,...,20.0,10,50.0,90,-,-,-,-,-,Eagle XG
5,01027,16_Feb_2016,BZC91,-,BZYF631,-,BZF19,-,BZC91/ BZF19,425.0,...,100.0,10,50.0,270,-,-,-,90,-,Eagle XG
6,01028,17_Feb_2016,BZC91,-,BZYF631,-,BZF19,-,BZC91/ BZF19,425.0,...,100.0,10,50.0,90,-,-,-,270,-,SiO2
7,01029,18_Feb_2016,BZC91,-,BZYF631,-,BZF19,-,BZC91/ BZYF631,425.0,...,100.0,10,50.0,90,-,270,-,-,-,SiO2
8,01030,18_Feb_2016,BZC91,-,BZYF631,-,BZF19,-,BZYF631/ BZF19,425.0,...,100.0,10,50.0,-,-,270,-,90,-,SiO2
9,01031,23_Feb_2016,BZC91,-,BZYF631,-,BZF19,-,BZC91/ BZF19,431.0,...,300.0,10,50.0,270,-,-,-,90,-,SiO2
